# Analyzing borrowers’ risk of defaulting

Our project is to prepare a report for a bank’s loan division. We’ll need to find out if a customers' marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

In [1]:
# Loading all the libraries
import pandas as pd
import numpy as np

# Loading the data
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [2]:
# Let's see how many rows and columns our dataset has
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# let's print the first 10 rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


From the printed data sample we see that some "days_employed" values have negative values, "education" column contains mixed case strings.

In [4]:
# getting info on data
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in 2 columns - "days_employed" and "total_income", and the number of missing values in both is the same, it seems we have a pattern here and need to find out more what's happening between these 2 columns.

In [5]:
# Let's look in the filtered table at the the first column with missing data
days_employed_na = df[df['days_employed'].isna()]
days_employed_na.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


From the filtered table the values do seem symmetric, but we cannot be sure as there are 2000 and + more rows.

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
days_employed_total_income_na = df[(df['days_employed'].isna()) & (df['total_income'].isna())]
days_employed_total_income_na
whole_dataset = df.size
missing_values = days_employed_total_income_na.size
missing_values_ratio = missing_values / whole_dataset
print('Missing values conversion rate: {:.0%}'.format(missing_values_ratio))


Missing values conversion rate: 10%


**Intermediate conclusion**

The number of rows in the filtered table match the number of missing values, which means missing values are symmetric.

Next I'll look at clients who have missing values in "days employed" and "total income" columns to see if there is any distinct characteristic in "income type" column.

In [8]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
days_employed_total_income_na[['income_type']].head()

,income_type
12,retiree
26,civil servant
29,retiree
41,civil servant
55,retiree


In [9]:
# checking distribution
print(df['income_type'].value_counts(normalize=True))
print()
print(days_employed_total_income_na['income_type'].value_counts(normalize=True))

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64


After looking at distribution of values in the "income type" column between the whole dataset and filtered dataset, we do not see any significant change or pattern. 

It does look like the values are missing randomly.

In [10]:
# checking the distribution in the whole dataset
print(df['income_type'].value_counts(normalize=True))

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64


In [12]:
# checking for other patterns
print(df['dob_years'].value_counts())
print(days_employed_total_income_na['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65 

No patterns in missing values found.

## Data transformation

In [13]:
# let's see all values in education column to check if and what spellings will need to be fixed
sorted(df['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [14]:
# converting education column values to lowercase
df['education'] = df['education'].str.lower()

In [15]:
# checking all the values in the column to make sure we fixed them
sorted(df['education'].unique())

["bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [16]:
# let's see the distribution of values in the `children` column
print(df['children'].value_counts(normalize=True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


In [17]:
# replacing values in cells where the number of children are 20 and -1 to 'nan' as this data is unclear
df['children'] = df['children'].replace([20,-1], np.nan)

In [18]:
# checking the `children` column again to make sure it's all fixed
df['children'].unique()

array([ 1.,  0.,  3.,  2., nan,  4.,  5.])

In [19]:
# finding problematic data in `days_employed` and calculating the percentage
negative_values = df['days_employed'] < 0
nan_values = df['days_employed'].isna()
print((nan_values | negative_values).value_counts(normalize=True))
#print(df['days_employed'].isna().value_counts())

True     0.839954
False    0.160046
Name: days_employed, dtype: float64


In [20]:
# replacing negative values in days_employed with positive values
# as they appear to be caused by system error
df['days_employed'] = df['days_employed'].abs()

In [21]:
# checking the result
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [22]:
# checking the `dob_years` for suspicious values and counting the percentage
print(df['dob_years'].value_counts(normalize=True))

35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
66    0.008502
22    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64


Replacing age 0 to nan as the reason for this value in age column is unclear.

In [23]:
# addressing the issues in the `dob_years` column
df['dob_years'] = df['dob_years'].replace([0], np.nan)

In [24]:
# checking the result
(df['dob_years'] == 0).value_counts()

False    21525
Name: dob_years, dtype: int64

In [25]:
# let's see the values in the family_status column
df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [26]:
# let's see the values in the gender column
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [27]:
# checking the row where we have 'XNA'
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0.0,2358.600502,24.0,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


In [28]:
# let's see the values in the income_type column
df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [29]:
# checking duplicates
df.duplicated().sum()

71

In [30]:
# removing the duplicates
df = df.drop_duplicates().reset_index(drop = True)

In [31]:
# checking the result
df.duplicated().sum()

0

In [32]:
# checking the size of the dataset that we now have after the first manipulations with it
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  float64
 1   days_employed     19351 non-null  float64
 2   dob_years         21353 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.0+ MB


# Working with missing values

### Restoring missing values in `total_income`

In [33]:
# Let's write a function that calculates the age category
def age_group(row):
    age = row['dob_years']
    
    if age < 15:
        return 'na'

    if age <= 24:
        return 'youth'

    if age <= 64:
        return 'adult'

    return 'senior'

    

In [34]:
# testing whether the function does work
row_values = [19]
row_columns = ['dob_years']
row = pd.Series(data=row_values, index=row_columns) 
age_group(row)

'youth'

In [35]:
# creating new column based on function
df['age_group'] = df.apply(age_group, axis=1)

In [36]:
# checking values in the new column
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1.0,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1.0,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0.0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3.0,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0.0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult


In [37]:
# creating a table without missing values and printing a few of its rows to make sure it looks fine
df_wo_nan = df.dropna()
df_wo_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1.0,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1.0,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0.0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3.0,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0.0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult


In [38]:
# looking at the mean values for income based on age group
avg_income = df_wo_nan.groupby('age_group').mean()
avg_income['total_income']

age_group
adult     27222.746837
senior    21545.431431
youth     22729.843098
Name: total_income, dtype: float64

In [39]:
# looking at the median values for income based on age group
median_income = df_wo_nan.groupby('age_group').median()
median_income['total_income']

age_group
adult     23546.495
senior    18425.306
youth     20577.582
Name: total_income, dtype: float64

In [40]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21331.000000,19351.000000,21353.000000,21454.000000,21454.000000,21454.000000,19351.000000
mean,0.474005,66914.728907,43.475905,0.817097,0.973898,0.081150,26787.568355
std,0.752378,139030.880527,12.242316,0.548674,1.421567,0.273072,16475.450632
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,32549.611000
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Upon looking at total_income values, it appears there are outliers and distribution is not symmetric. Using mean values would be an overestimate. It's better to fill missing data in total_income with median total income values depending on age group.

In [41]:
#  writing a function for filling in missing values in total_income with median values
def total_income_fillna(row):
    age_group = row['age_group']
    if age_group == 'youth' and pd.isnull(row['total_income']):
        return 20577.582
    if age_group == 'adult' and pd.isnull(row['total_income']):
        return 23546.495
    if age_group == 'senior' and pd.isnull(row['total_income']):
        return 18425.306
    if not pd.isnull(row['total_income']):
        return row ['total_income']

In [42]:
# checking whether the function does work
row_values = ['senior', 2]
row_columns = ['age_group', 'total_income']
row = pd.Series(data=row_values, index=row_columns) 
total_income_fillna(row)

2

In [43]:
# applying it to every row
df['filled_income'] = df.apply(total_income_fillna, axis = 1)

In [44]:
# checking whether we got any errors
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  float64
 1   days_employed     19351 non-null  float64
 2   dob_years         21353 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
 13  filled_income     21454 non-null  float64
dtypes: float64(5), int64(3), object(6)
memory usage: 2.3+ MB


In [45]:
# replacing missing values in total income with median total income values depending on age group
df['total_income'] = df['filled_income']

In [46]:
# checking the number of entries in the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  float64
 1   days_employed     19351 non-null  float64
 2   dob_years         21353 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
 13  filled_income     21454 non-null  float64
dtypes: float64(5), int64(3), object(6)
memory usage: 2.3+ MB


###  Restoring values in `days_employed`

In [47]:
# checking the distribution of `days_employed` medians based on income type
days_employed_median = df.pivot_table(index='income_type', values='days_employed', aggfunc=['count', 'median'])
days_employed_median

,count,median
,days_employed,days_employed
income_type,,
business,4577,1547.382223
civil servant,1312,2689.368353
employee,10014,1574.202821
entrepreneur,1,520.848083
paternity / maternity leave,1,3296.759962
retiree,3443,365213.306266
student,1,578.751554
unemployed,2,366413.652744


In [48]:
# checking distribution of `days_employed` means based on income type
days_employed_mean = df.pivot_table(index='income_type', values='days_employed', aggfunc=['count', 'mean'])
days_employed_mean

,count,mean
,days_employed,days_employed
income_type,,
business,4577,2111.524398
civil servant,1312,3399.896902
employee,10014,2326.499216
entrepreneur,1,520.848083
paternity / maternity leave,1,3296.759962
retiree,3443,365003.491245
student,1,578.751554
unemployed,2,366413.652744


In [49]:
# days_employed that are higher than 60 years are going to be replaced by NaN to avoid taking them
# into account when calculating medians for income type categories
df.loc[df['days_employed'] > 21900, 'days_employed'] = np.nan
days_employed_median = df.pivot_table(index='income_type', values='days_employed', aggfunc='median')
days_employed_median

,days_employed
income_type,
business,1547.382223
civil servant,2689.368353
employee,1574.202821
entrepreneur,520.848083
paternity / maternity leave,3296.759962
student,578.751554


In [50]:
df['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

Since the distribution in days_employed is skewed, I will fill missing values with median values in days_employed based on income_type categories.

I will also fill retiree and unemployed income_type categories with median values across days_employed column.

In [51]:
overall_days_employed_median = df['days_employed'].median()
overall_days_employed_median

1630.0193809778218

In [52]:
# writing a function for filling in missing values in days_employed with median values
def days_employed_fillna(row):
    income_type = row['income_type']

    if pd.isnull(row['days_employed']):
        if income_type in days_employed_median.index:
            return days_employed_median.loc[income_type, "days_employed"]
        else:
            return overall_days_employed_median
    else:
        return row['days_employed']

In [53]:
# making sure that the function does work
row_values = ['retiree', np.nan]
row_columns = ['income_type', 'days_employed']
row = pd.Series(data=row_values, index=row_columns) 
days_employed_fillna(row)

1630.0193809778218

In [54]:
# creating a new column based on function
df['filled_days_employed'] = df.apply(days_employed_fillna, axis = 1)

In [55]:
# checking the general info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21331 non-null  float64
 1   days_employed         15906 non-null  float64
 2   dob_years             21353 non-null  float64
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          21454 non-null  float64
 11  purpose               21454 non-null  object 
 12  age_group             21454 non-null  object 
 13  filled_income         21454 non-null  float64
 14  filled_days_employed  21454 non-null  float64
dtypes: float64(6), int6

## Categorization of data

In [56]:
df.select_dtypes(include=['object']).head()

,education,family_status,gender,income_type,purpose,age_group
0,bachelor's degree,married,F,employee,purchase of the house,adult
1,secondary education,married,F,employee,car purchase,adult
2,secondary education,married,M,employee,purchase of the house,adult
3,secondary education,married,M,employee,supplementary education,adult
4,secondary education,civil partnership,F,retiree,to have a wedding,adult


In [57]:
# checking the unique values in purpose column
sorted(df['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

In [58]:
# let's write a function to categorize the data based on common topics
def purpose(row):
    cat = row['purpose']
    
    if 'wedding' in row['purpose']:
        return 'wedding'
    if 'car' in row['purpose']:
        return 'car'
    if 'education' in row['purpose'] or 'university' in row['purpose']:
        return 'education'
    
    return 'real estate'

In [59]:
# applying the function and filling values in purpose column
df['purpose'] = df.apply(purpose, axis=1)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,filled_income,filled_days_employed
0,1.0,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,real estate,adult,40620.102,8437.673028
1,1.0,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,adult,17932.802,4024.803754
2,0.0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,real estate,adult,23341.752,5623.422610
3,3.0,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,adult,42820.568,4124.747207
4,0.0,NaN,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,adult,25378.572,1630.019381


## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [60]:
# checking the children data and paying back on time
pivot_1 = df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_1

,count,sum,mean
,debt,debt,debt
children,,,
0.0,14091,1063,0.075438
1.0,4808,444,0.092346
2.0,2052,194,0.094542
3.0,330,27,0.081818
4.0,41,4,0.097561
5.0,9,0,0.000000


**Conclusion**:
There's a slight correlation between amount of children and paying back on time. It appears, people who have no children are less likely to have debts than people who have 1 or 2 children.

**Is there a correlation between family status and paying back on time?**

In [61]:
# checking the family status data and paying back on time
pivot_2 = df.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_2

,count,sum,mean
,debt,debt,debt
family_status,,,
civil partnership,4151,388,0.093471
divorced,1195,85,0.071130
married,12339,931,0.075452
unmarried,2810,274,0.097509
widow / widower,959,63,0.065693


**Conclusion**: There's also a slight correlation between family status and paying back on time. It appears, those who are married are less likely to have debts than those who are unmarried or in a civil partnership.

**Is there a correlation between income level and paying back on time?**

In [62]:
# checking the income level data and paying back on time
# <creating the function income_cat()>

def income_cat(row):
    '''
    Categorizes loan requests by income level
    Parameters:
        row: the row to categorize
    '''

    income = row['total_income']
    
    if income < 2666:
        return 'low income'

    if income <= 4450:
        return 'lower middle'

    if income <= 8900:
        return 'middle class'
    
    if income <= 31000:
        return 'upper middle'

    return 'high income'

In [63]:
# testing the function
row_values = [31100]
row_columns = ['total_income']
row = pd.Series(data=row_values, index=row_columns) 
income_cat(row)

'high income'

In [64]:
# creating new column based on function
df['income_cat'] = df.apply(income_cat, axis=1)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,filled_income,filled_days_employed,income_cat
0,1.0,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,real estate,adult,40620.102,8437.673028,high income
1,1.0,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,adult,17932.802,4024.803754,upper middle
2,0.0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,real estate,adult,23341.752,5623.422610,upper middle
3,3.0,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,adult,42820.568,4124.747207,high income
4,0.0,NaN,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,adult,25378.572,1630.019381,upper middle


In [65]:
# calculating default-rate based on income level
pivot_3 = df.pivot_table(index='income_cat', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_3

,count,sum,mean
,debt,debt,debt
income_cat,,,
high income,5501,396,0.071987
lower middle,14,1,0.071429
middle class,603,37,0.061360
upper middle,15336,1307,0.085224


# Calculating default-rate based on income level


**Conclusion**: We can observe a correlation between level of income and paying back on time. We see that middle class population are less likely to have debts than upper middle or high income populations.

**How does credit purpose affect the default rate?**

In [66]:
# checking the percentages for default rate for each credit purpose and analyzing them
pivot_4 = df.pivot_table(index='purpose', values='debt', aggfunc=['count', 'sum', 'mean'])
pivot_4

,count,sum,mean
,debt,debt,debt
purpose,,,
car,4306,403,0.093590
education,3605,331,0.091817
real estate,11219,821,0.073179
wedding,2324,186,0.080034


**Conclusion**: We can observe customers' whose credit purpose is related to real estate are less likely to have debts than customers who are taking loan for car purchase or education.

# General Conclusion 

The dataset provided had several issues that needed to be addressed:

1) There were missing values in 2 columns - "days_employed" and "total_income" that made up 10% of dataset. No distinct characteristics for the rows with missing values were found.

2) There is 84% of problematic data in days employed column - negative value numbers or missing values. Since the amount of problematic data in this column is very high, this column has been left untouched and the values remain as they originally were.

3) Replacing values in cells where the number of children are 20 and -1 to 'nan' as the reason for these values is unclear.

4) Replacing age 0 in dob_years column to nan as the reason for this value in age column is unclear.

5) Created credit purpose categories, narrowed down to - car, education, real estate, wedding. Replaced previous column values to these categories.

6) Added "age_group" column that includes age categories - youth, adult and senior.

7) Added "filled_income" column where missing values in "total_income" column are filled with median total income values depending on age group.

8) Added "filled_days_employed" column where missing values in "days_employed" column are filled with median days employed values depending on income type.

9) Made categories based on "total_income" column and added "income_ca" column that includes income categories - low income, lower middle, middle class, upper middle, high income.


There's a slight correlation between amount of children and paying back on time. It appears, people who have no children are less likely to have debts than people who have 1 or 2 children.

There's also a slight correlation between family status and paying back on time. It appears, those who are married are less likely to have debts than those who are unmarried or in a civil partnership.

We can observe a correlation between level of income and paying back on time. We see that middle class population are less likely to have debts than upper middle or high income populations.

We also see customers' whose credit purpose is related to real estate are less likely to have debts than customers who are taking loan for car purchase or education.